In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

import pickle # to save/load objects
import os
import sys
sys.path.append("../helper/") # go to parent dir
from functions_preprocess import *
from functions_model import *

%load_ext autoreload
%autoreload 2

### Data Preprocessing
Here, I'm getting, cleaning and processing the data. I intentionally mixed the bands with different characteristics. I thought this may prevent overfitting and provide better word pool. Data has the almost entire discography of the bands, even demos for some.  

I'm also creating two dictionaries here, `vocab_to_int` and `int_to_vocab`. The model need numerical representation of the characters to be able to compute weights, biases, etc.

In [2]:
# prepare training data
artist_list = next(os.walk('../data/'))[1] # just need folder names, each named folders has albums

text = ''
folder_name = ''
for i in artist_list:
    text = text + combine_songs(i)
    folder_name = folder_name + i[:1] # I know this's not the greatest folder name
    
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
chars = np.array([vocab_to_int[c] for c in text], dtype=np.int32)
print(folder_name, "ready with", len(chars), "chars")

aaaaabbcdddeeefhiiijkkmmmmnnoopssssssttw ready with 6292269 chars


### Saving
I need to save the objects I created so far. Remember, no orders in Python dictionaries. This means this notebook creates different dictionaries in each session. Because I need the use my model later, I have to save these lookup dictionaries.

In [3]:
# Saving the objects
directory = 'checkpoints/{}'.format(folder_name)
if not os.path.exists(directory): os.makedirs(directory) # create folder first
f = open('checkpoints/{}/vars.pckl'.format(folder_name), 'wb')
pickle.dump([vocab, vocab_to_int, int_to_vocab, artist_list], f, protocol=2)
f.close()

In [4]:
# hyperparameters
batch_size = 100
num_steps = 100
lstm_size = 512
num_layers = 2
learning_rate = 0.001
keep_prob_train = 0.7

I highly encourage you to check the function `split_data` in `functions_model.py`. The target is the next character in the sequence.  
`x = chars[: n_batches*slice_size]`  
`y = chars[1: n_batches*slice_size + 1]`

Here is the main graph of the following model from <a href="https://www.tensorflow.org/get_started/summaries_and_tensorboard" target="_blank">TensorBoard</a>,
<img src="assets/main_graph.png" width="600">

In [5]:
start_time = time.time() # always measure the time
epochs = 100
save_every_n = 2000
# Split training and validation sets
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps) # default fraction for trainning is 0.9

model = build_rnn(len(vocab), 
                  batch_size=batch_size,
                  num_steps=num_steps,
                  learning_rate=learning_rate,
                  lstm_size=lstm_size,
                  num_layers=num_layers)

saver = tf.train.Saver(max_to_keep=100)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('./logs/{}/train'.format(folder_name), sess.graph)
    test_writer = tf.summary.FileWriter('./logs/{}/test'.format(folder_name))
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/lyr20.ckpt')
    
    n_batches = int(train_x.shape[1]/num_steps)
    iterations = n_batches * epochs
    for e in range(epochs):
        
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for b, (x, y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
            iteration = e*n_batches + b
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob_train,
                    model.initial_state: new_state}
            summary, batch_loss, new_state, _ = sess.run([model.merged, model.cost, 
                                                          model.final_state, model.optimizer], 
                                                          feed_dict=feed)
            loss += batch_loss
            end = time.time()
            print('Epoch {}/{} '.format(e+1, epochs),
                  'Iteration {}/{}'.format(iteration, iterations),
                  'Training loss: {:.4f}'.format(loss/b),
                  '{:.4f} sec/batch'.format((end-start)))
            
            train_writer.add_summary(summary, iteration)
        
            if (iteration%save_every_n == 0) or (iteration == iterations):
                # Check performance, notice dropout has been set to 1 
                # because this is validation, not training, so we need everything
                val_loss = []
                new_state = sess.run(model.initial_state)
                for x, y in get_batch([val_x, val_y], num_steps):
                    feed = {model.inputs: x,
                            model.targets: y,
                            model.keep_prob: 1.,
                            model.initial_state: new_state}
                    summary, batch_loss, new_state = sess.run([model.merged, model.cost, 
                                                               model.final_state], feed_dict=feed)
                    val_loss.append(batch_loss)
                    
                test_writer.add_summary(summary, iteration)

                print('Validation loss:', np.mean(val_loss),
                      'Saving checkpoint!')
                saver.save(sess, "checkpoints/{}/i{}_l{}_{:.3f}.ckpt".format(folder_name, iteration, lstm_size, np.mean(val_loss)))

print(epochs, "epoch done in", round(time.time() - start_time, 2)/60, "mins")

Epoch 1/100  Iteration 1/56600 Training loss: 4.4892 0.8543 sec/batch
Epoch 1/100  Iteration 2/56600 Training loss: 4.4489 0.4667 sec/batch
Epoch 1/100  Iteration 3/56600 Training loss: 4.2830 0.4863 sec/batch
Epoch 1/100  Iteration 4/56600 Training loss: 4.4983 0.4636 sec/batch
Epoch 1/100  Iteration 5/56600 Training loss: 4.4387 0.4756 sec/batch
Epoch 1/100  Iteration 6/56600 Training loss: 4.3224 0.4835 sec/batch
Epoch 1/100  Iteration 7/56600 Training loss: 4.2203 0.4727 sec/batch
Epoch 1/100  Iteration 8/56600 Training loss: 4.1332 0.4716 sec/batch
Epoch 1/100  Iteration 9/56600 Training loss: 4.0553 0.4653 sec/batch


Epoch 87/100  Iteration 48899/56600 Training loss: 0.9747 0.4630 sec/batch
Epoch 87/100  Iteration 48900/56600 Training loss: 0.9748 0.4683 sec/batch
Epoch 87/100  Iteration 48901/56600 Training loss: 0.9750 0.4738 sec/batch
Epoch 87/100  Iteration 48902/56600 Training loss: 0.9751 0.4800 sec/batch
Epoch 87/100  Iteration 48903/56600 Training loss: 0.9753 0.4710 sec/batch
Epoch 87/100  Iteration 48904/56600 Training loss: 0.9752 0.4912 sec/batch
Epoch 87/100  Iteration 48905/56600 Training loss: 0.9752 0.4788 sec/batch
Epoch 87/100  Iteration 48906/56600 Training loss: 0.9754 0.4788 sec/batch
Epoch 87/100  Iteration 48907/56600 Training loss: 0.9756 0.4905 sec/batch
Epoch 87/100  Iteration 48908/56600 Training loss: 0.9757 0.4844 sec/batch


Epoch 100/100  Iteration 56585/56600 Training loss: 0.9589 0.4788 sec/batch
Epoch 100/100  Iteration 56586/56600 Training loss: 0.9589 0.4794 sec/batch
Epoch 100/100  Iteration 56587/56600 Training loss: 0.9589 0.4902 sec/batch
Epoch 100/100  Iteration 56588/56600 Training loss: 0.9589 0.4791 sec/batch
Epoch 100/100  Iteration 56589/56600 Training loss: 0.9588 0.4787 sec/batch
Epoch 100/100  Iteration 56590/56600 Training loss: 0.9588 0.4844 sec/batch
Epoch 100/100  Iteration 56591/56600 Training loss: 0.9588 0.4716 sec/batch
Epoch 100/100  Iteration 56592/56600 Training loss: 0.9589 0.4811 sec/batch
Epoch 100/100  Iteration 56593/56600 Training loss: 0.9589 0.4704 sec/batch
Epoch 100/100  Iteration 56594/56600 Training loss: 0.9589 0.4692 sec/batch
Epoch 100/100  Iteration 56595/56600 Training loss: 0.9590 0.4716 sec/batch
Epoch 100/100  Iteration 56596/56600 Training loss: 0.9590 0.4817 sec/batch
Epoch 100/100  Iteration 56597/56600 Training loss: 0.9590 0.4730 sec/batch
Epoch 100/10

Here is the loss graph,
<img src="assets/loss_graph2.JPG" width="800">

### Test
No test here, because I'm not predicting house prices. The model is to generate, not to predict. However, the validation loss is a good indicater about how well the model behaves. And, looks like I already got the best model almost in the middle, I waited for ~200 minutes just to know no need go more.

In [7]:
# list all checkpoints
tf.train.get_checkpoint_state('checkpoints/{}'.format(folder_name))

model_checkpoint_path: "checkpoints/aaaaabbcdddeeefhiiijkkmmmmnnoopssssssttw\\i56600_l512_1.206.ckpt"
all_model_checkpoint_paths: "checkpoints/aaaaabbcdddeeefhiiijkkmmmmnnoopssssssttw\\i2000_l512_1.425.ckpt"
all_model_checkpoint_paths: "checkpoints/aaaaabbcdddeeefhiiijkkmmmmnnoopssssssttw\\i4000_l512_1.303.ckpt"
all_model_checkpoint_paths: "checkpoints/aaaaabbcdddeeefhiiijkkmmmmnnoopssssssttw\\i6000_l512_1.260.ckpt"
all_model_checkpoint_paths: "checkpoints/aaaaabbcdddeeefhiiijkkmmmmnnoopssssssttw\\i8000_l512_1.236.ckpt"
all_model_checkpoint_paths: "checkpoints/aaaaabbcdddeeefhiiijkkmmmmnnoopssssssttw\\i10000_l512_1.227.ckpt"
all_model_checkpoint_paths: "checkpoints/aaaaabbcdddeeefhiiijkkmmmmnnoopssssssttw\\i12000_l512_1.218.ckpt"
all_model_checkpoint_paths: "checkpoints/aaaaabbcdddeeefhiiijkkmmmmnnoopssssssttw\\i14000_l512_1.213.ckpt"
all_model_checkpoint_paths: "checkpoints/aaaaabbcdddeeefhiiijkkmmmmnnoopssssssttw\\i16000_l512_1.211.ckpt"
all_model_checkpoint_paths: "checkpoints/aaaaa

In [8]:
checkpoint = "checkpoints/{}/i34000_l512_1.195.ckpt".format(folder_name)
samp = sample(checkpoint, 700, lstm_size, len(vocab), vocab_to_int, vocab, int_to_vocab, prime="The ", top_n=4)
print(samp)

The Stench Of With Hell

I am your land
I am the path to save me
The sorrow, I hear your fate
I will never know,
I cannot hope you I will die again
The one to the other side

As I still defend the writings
On my soul to search for me
And in a dream I'm the masquerade
This is the final scene

Too many stories of the lies
They're too many shouts to see
They set in our holy lead
And the checks of summer waits

The chosen cast in soul of an our own divide
The stench of far to stark on far away
And who's a thousand to be free again

When all the thoughts won't haunt me
And I should be trying to give up
And then this is my last
I can never see
And now I feel so far
They come along the way
They will sur


### Source
- [Udacity Deep Learning](https://github.com/udacity/deep-learning)  
- [Mat Leonard](https://github.com/mcleonard)